In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import no_op, fast_dsift
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images
from menpo.image import Image

## Load images

In [ ]:
images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=10):
    if i.n_channels == 3:
        i = i.crop_to_landmarks_proportion(0.8)
        i = i.rescale_landmarks_to_diagonal_range(200)
        labeller(i, 'PTS', ibug_face_66)
        images.append(i)

In [ ]:
visualize_images(images)

In [ ]:
shapes = [i.landmarks['ibug_face_66'].lms for i in images]

In [ ]:
from menpo.shape import PointCloud

landmarks = [PointCloud(s.points[40:42,:])  for s in shapes]

In [ ]:
from menpofit.clm.patch_experts import LinearSVMExpertEnsemble
from menpo.shape import PointCloud

cfee = LinearSVMExpertEnsemble(
    images, 
    landmarks, 
    verbose=True,
    positive_neighbourhood=(5, 5),
    patch_size=(33, 33),
    context_size=(33, 33),
    negative_step=(3, 3),
    sample_offsets=None)

In [ ]:
visualize_images(cfee.spatial_filter_images)

In [ ]:
responses = []
for (image, landmark) in zip(images, landmarks):
    a = cfee.predict_normalise(image, landmark)[1]
    response = Image(a)
    responses.append(response)
    
visualize_images(responses)